# Recommender Systems with DGL

## Introduction

Graph Neural Networks (GNN), as a methodology of learning representations on graphs, has gained much attention recently.  Various models such as Graph Convolutional Networks, GraphSAGE, etc. are proposed to obtain representations of whole graphs, or nodes on a single graph.

A primary goal of Collaborative Filtering (CF) is to automatically make predictions about a user's interest, e.g. whether/how a user would interact with a set of items, given the interaction history of the user herself, as well as the histories of other users.  The user-item interaction can also be viewed as a bipartite graph, where users and items form two sets of nodes, and edges connecting them stands for interactions.  The problem can then be formulated as a *link-prediction* problem, where we try to predict whether an edge (of a given type) exists between two nodes.

Based on this intuition, the academia developed multiple new models for CF, including but not limited to:

* Geometric Learning Approaches
  * [Geometric Matrix Completion](https://papers.nips.cc/paper/5938-collaborative-filtering-with-graph-information-consistency-and-scalable-methods.pdf)
  * [Recurrent Multi-graph CNN](https://arxiv.org/pdf/1704.06803.pdf)
* Graph-convolutional Approaches
  * Models such as [R-GCN](https://arxiv.org/pdf/1703.06103.pdf) or [GraphSAGE](https://github.com/stellargraph/stellargraph/tree/develop/demos/link-prediction/hinsage) also apply.
  * [Graph Convolutional Matrix Completion](https://arxiv.org/abs/1706.02263)
  * [PinSage](https://arxiv.org/pdf/1806.01973.pdf)
  
In this hands-on tutorial, we will demonstrate how to write GraphSAGE in DGL + MXNet, and how to apply it in a recommender system setting.

## Dependencies

* Latest DGL release: `conda install -c dglteam dgl`
* `pandas`
* `stanfordnlp`
* `mxnet`
* `tqdm` for displaying the progress bar.

## Loading data

In this tutorial, we focus on rating prediction on MovieLens-1M dataset.  The data comes from [MovieLens](http://files.grouplens.org/datasets/movielens/ml-1m.zip) and is shipped with the notebook already.

After loading and train-validation-test-splitting the dataset, we process the movie title into (padded) word-ID sequences, and other features into categorical variables (i.e. integers).  We then store them as node features on the graph.

Since user features and item features are different, we pad both types of features with zeros.

All of the above is encapsulated in `movielens.MovieLens` class for clarity of this notebook.

In [ ]:
import mxnet as mx
from mxnet import ndarray as nd, autograd, gluon
from mxnet.gluon import nn
import dgl
import dgl.function as FN
import numpy as np

In [ ]:
import movielens
import stanfordnlp

# IMPORTANT!!!
# If you don't have stanfordnlp installed and the English models downloaded, please uncomment this statement
#stanfordnlp.download('en', force=True)

ml = movielens.MovieLens('ml-100k')

## See the features in the MovieLens dataset

The MovieLens dataset has some user features and movie features.

User features:
* age,
* gender,
* occupation,
* zip code,

Movie features:
* genre,
* year,
* title

We use one-hot encoding for "age", "gender", "occupation", "zip code" and "year". "genre" uses multi-hop encoding while "title" encodes the frequency of different words. For simplicity, we store "genre" and "title" in float32 dense matrices.

In [ ]:
g = ml.g
print(g.ndata)

MovieLens is a bipartite graph. It has user nodes and movie nodes. When we construct the graph, we add a vector on the node data to identify the node type of every node. User nodes are `1` and movie nodes are `0`.

In [ ]:
g.ndata['type']

## Compute embeddings on the MovieLens dataset

"age", "gender", "occupation", "zip code" and "year" use one-hot encoding. "genre" and "title" are stored in float32 dense matrices. In addition, we add one-hot encoding for every user node and every movie node.

We add them to construct the inital node features.

In [ ]:
class MovieLensEmbedding(nn.Block):
    def __init__(self, G, feature_size):
        super(MovieLensEmbedding, self).__init__()
        
        # Embedding matrices for one-hot encoding.
        self.emb_age = nn.Embedding(G.ndata['age'].max().asscalar() + 1,
                                    feature_size)
        self.emb_gender = nn.Embedding(G.ndata['gender'].max().asscalar() + 1,
                                       feature_size)
        self.emb_occupation = nn.Embedding(G.ndata['occupation'].max().asscalar() + 1,
                                           feature_size)
        self.emb_zip = nn.Embedding(G.ndata['zip'].max().asscalar() + 1,
                                    feature_size)
        self.emb_year = nn.Embedding(G.ndata['year'].max().asscalar() + 1,
                                     feature_size)

        # Linear projection for float32 features.
        seq = nn.Sequential()
        with seq.name_scope():
            seq.add(nn.Dense(feature_size))
            seq.add(nn.LeakyReLU(0.1))
        self.proj_genre = seq

        seq = nn.Sequential()
        with seq.name_scope():
            seq.add(nn.Dense(feature_size))
            seq.add(nn.LeakyReLU(0.1))
        self.proj_title = seq

        # One-hot encoding for each node.
        self.node_emb = nn.Embedding(G.number_of_nodes() + 1, feature_size)

    def forward(self, ndata, nid):
        ndata['h'] = self.node_emb(nid + 1)
        
        extra_repr = []
        #if is_user:
        extra_repr.append(self.emb_age(ndata['age']))
        extra_repr.append(self.emb_gender(ndata['gender']))
        extra_repr.append(self.emb_occupation(ndata['occupation']))
        extra_repr.append(self.emb_zip(ndata['zip']))
        #else:
        extra_repr.append(self.emb_year(ndata['year']))
        extra_repr.append(self.proj_genre(ndata['genre']))
        extra_repr.append(self.proj_title(ndata['title']))
        ndata['h'] = ndata['h'] + nd.stack(*extra_repr, axis=0).sum(axis=0)

## Model

We can now write a GraphSAGE layer.  In GraphSAGE, the node representation is updated with the representation in the previous layer as well as an aggregation (often mean) of "messages" sent from all neighboring nodes.

### Algorithm

The algorithm of a single GraphSAGE layer goes as follows for each node $v$:

1. $h_{\mathcal{N}(v)} \gets \mathtt{Average}_{u \in \mathcal{N}(v)} h_{u}$
2. $h_{v} \gets \sigma\left(W \cdot \mathtt{CONCAT}(h_v, h_{\mathcal{N}(v)})\right)$
3. $h_{v} \gets h_{v} / \lVert h_{v} \rVert_2$

where

* $\mathtt{Average}$ can be replaced by any kind of aggregation including `sum`, `max`, or even an LSTM.
* $\sigma$ is any non-linearity function (e.g. `LeakyReLU`)

We simply repeat the computation above for multiple GraphSAGE layers.

## GraphSage node update function

The MovieLens dataset has two types of nodes: users and movies. We need to perform separate node update functions on the two types of nodes.

The node update function performs the computation of the last two steps as shown above.

For the movie nodes,

$h_{m} \gets \sigma\left(W0 \cdot \mathtt{CONCAT}(h_m, h_{\mathcal{N}(m)} / d_{\mathcal{N}(m)})\right)$, 
$h_{m} \gets h_{m} / \lVert h_{m} \rVert_2$

For the user nodes,

$h_{u} \gets \sigma\left(W1 \cdot \mathtt{CONCAT}(h_u, h_{\mathcal{N}(u)} / d_{\mathcal{N}(u)})\right)$,
$h_{u} \gets h_{u} / \lVert h_{u} \rVert_2$

In [ ]:
class GraphSageConvWithSampling(nn.Block):
    def __init__(self, feature_size):
        super(GraphSageConvWithSampling, self).__init__()

        self.feature_size = feature_size
        self.W0 = nn.Dense(feature_size)
        self.W1 = nn.Dense(feature_size)
        self.leaky_relu = nn.LeakyReLU(0.1)

    def forward(self, nodes):
        # Node embedding from the previous layer.
        h = nodes.data['h']
        # Aggregation of the node embeddings in the neighborhood
        h_agg = nodes.data['h_agg']
        # Degree of the vertex.
        deg = nodes.data['deg'].expand_dims(1)
        h_concat = nd.concat(h, h_agg / nd.maximum(deg, 1e-6), dim=1)
        
        # There are two types of nodes. Each type should have their own model weights.
        h_new0 = self.W0(h_concat)
        h_new1 = self.W1(h_concat)
        # We need to pick the right embedding
        h_new = nd.where(nodes.data['type'], h_new0, h_new1)
        
        h_new = self.leaky_relu(h_new)
        # Layer norm
        return {'h': h_new / nd.maximum(h_new.norm(axis=1, keepdims=True), 1e-6)}

## GraphSage with sampling

When the graph scales up, the GraphSage update on the full graph becomes impractical, because the node embeddings couldn't fit in the GPU memory.

A natural solution would be partitioning the nodes and computing the embeddings one partition (minibatch) at a time.  The nodes at one convolution layer only depends on their neighbors, rather than all the nodes in the graph, hence reducing the computational cost.  However, if we have multiple layers, and some of the nodes have a lot of neighbors (which is often the case since the degree distribution of many real-world graphs follow [power-law](https://en.wikipedia.org/wiki/Scale-free_network)), computing the embedding of a target node still depends on a large number of nodes in the graph.


The data and computation dependency of computing the embedding on target node 1 is illustrated in the figure below:
<img src="https://s3.us-east-2.amazonaws.com/dgl.ai/amlc_tutorial/Dependency.png" width="400">

*Neighbor sampling* is an answer to further reduce the cost of computing node embeddings.  When aggregating messages, instead of collecting from all neighboring nodes, we only collect from some of the randomly-sampled (for instance, uniform sampling at most K neighbors without replacement) neighbors.

<img src="https://s3.us-east-2.amazonaws.com/dgl.ai/amlc_tutorial/neighbor_sampling.png" width="600">

DGL provides `NodeFlow` that stores the computation dependency of nodes in a graph convolutional network. Below shows hwo we can run GraphSage on `NodeFlow`.

In [ ]:
class GraphSageWithSampling(nn.Block):
    def __init__(self, feature_size, n_layers, G):
        super(GraphSageWithSampling, self).__init__()
        
        self.feature_size = feature_size
        self.n_layers = n_layers

        # Simulating ModuleList
        for i in range(n_layers):
            setattr(self, 'conv_%d' % i, GraphSageConvWithSampling(feature_size))

        self.G = G
        self.emb = MovieLensEmbedding(G, feature_size)

    msg = [FN.copy_src('h', 'h'), FN.copy_src('one', 'one')]
    red = [FN.sum('h', 'h_agg'), FN.sum('one', 'deg')]

    def forward(self, nf):
        '''
        nf: NodeFlow.
        '''
        nf.copy_from_parent(edge_embed_names=None)
        for i in range(nf.num_layers):
            self.emb(nf.layers[i].data, nf.layer_parent_nid(i))
            nf.layers[i].data['one'] = nd.ones(nf.layer_size(i))
            
        if self.n_layers == 0:
            return nf.layers[i].data['h']
        for i in range(self.n_layers):
            nf.block_compute(i, self.msg, self.red, getattr(self, 'conv_%d' % i))

        result = nf.layers[self.n_layers].data['h']
        assert (result != result).sum() == 0
        return result

## Edge sampling for rating prediction

For rating prediction, we first need to sample a set of edges. On the endpoint nodes of the edges, we run GraphSage to compute their embeddings. Therefore, we sample edges along with `NodeFlow`s. For each batch of sampled edges, we construct a `NodeFlow` for the endpoint of each side. This is illustrated with the figure below:

<img src="https://s3.us-east-2.amazonaws.com/dgl.ai/amlc_tutorial/rating_pred.png" width="300">

DGL doesn't provide a builtin edge sampler (the future version of DGL will have such an edge sampler). We can use `NeighborSampler` to implement this edge sampler. When the edge sampler constructs a batch, it creates a `NodeFlow` for the endpoint nodes of both sides as well as endpoint nodes and ratings of the edges.

In [ ]:
batch_size = 1024                       # The number of target nodes in a batch.
num_neighbors = 5                       # The number of sampled neighbors on each node
num_layers = 1.                         # The number of layers in GraphSage.

class EdgeSampler:
    def __init__(self, g, src, dst, rating):
        shuffle_idx = nd.from_numpy(np.random.permutation(g.number_of_edges()))
        src_shuffled = src[shuffle_idx]
        dst_shuffled = dst[shuffle_idx]
        rating_shuffled = rating[shuffle_idx]
        
        self.src_batches = []
        self.dst_batches = []
        self.rating_batches = []
        for i in range(0, g.number_of_edges(), batch_size):
            j = min(i + batch_size, g.number_of_edges())
            self.src_batches.append(src_shuffled[shuffle_idx[i:j]])
            self.dst_batches.append(dst_shuffled[shuffle_idx[i:j]])
            self.rating_batches.append(rating_shuffled[shuffle_idx[i:j]])

        # HACK 2: Alternate between source batch and destination batch, so we can put exactly
        # a batch of edges' endpoints in a single NodeFlow.
        seed_nodes = nd.concat(*sum([[s, d] for s, d in zip(self.src_batches, self.dst_batches)], []), dim=0)

        self.sampler = iter(dgl.contrib.sampling.NeighborSampler(
            g,                     # the graph
            batch_size * 2,        # number of nodes to compute at a time, HACK 2
            num_neighbors,         # number of neighbors for each node
            num_layers,            # number of layers in GCN
            seed_nodes=seed_nodes, # list of seed nodes, HACK 2
            prefetch=True,         # whether to prefetch the NodeFlows
            add_self_loop=True,    # whether to add a self-loop in the NodeFlows, HACK 1
            shuffle=False,         # whether to shuffle the seed nodes.  Should be False here.
            num_workers=4,
        ))
        self.i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i == len(self.src_batches):
            raise StopIteration
            
        idx = self.i
        self.i += 1
        return (self.src_batches[idx], self.dst_batches[idx],
                self.rating_batches[idx], next(self.sampler))

## Rating score

In this graph, a movie node can only connect to a user mode, vice versa. There are no connections between movie nodes, nor between user nodes.

For recommendation, the rating on item $j$ by user $i$ is defined by $u_i^T v_j$.

We minimize $$\Sigma_{i,j}(r_{i,j}-(u_i^T v_j))^2$$.

In practice, recommendation models have user bias term and movie bias term. Thus, we minimize
$$\Sigma_{i,j}(r_{i,j}-(u_i^T v_j + b_{u_i} + b_{v_j}))^2$$.

When we generate a `NodeFlow`, roughly half of the target nodes are both movie nodes and half of them are user nodes. When we run GraphSage on the target nodes, we basically improve the embedding of the target nodes with their neighbors. Then we use the final node embedding for rating prediction.

In [ ]:
class GraphSAGERecommender(nn.Block):
    def __init__(self, gcn):
        super(GraphSAGERecommender, self).__init__()
        
        with self.name_scope():
            self.gcn = gcn
            self.node_biases = self.params.get(
                'node_biases',
                init=mx.init.Zero(),
                shape=(gcn.G.number_of_nodes()+1,))
        
    def forward(self, nf, src, dst):
        h_output = self.gcn(nf)
        h_src = h_output[nf.map_from_parent_nid(-1, src, True)]
        h_dst = h_output[nf.map_from_parent_nid(-1, dst, True)]
        score = (h_src * h_dst).sum(1) + self.node_biases.data()[src+1] + self.node_biases.data()[dst+1]
        return score

## Construct the training set

We train on a subset of edges in the MovieLens dataset. To construct the training set, we takes all the edges for training and construct a graph with these edges.

We first use `filter_edges` to select the edge Ids for training. We call `edge_subgraph` to construct the induced subgraph with the training edges. In the induced subgraph, we preserve all nodes from the parent graph.

In [ ]:
g = ml.g
# Find the subgraph of all "training" edges
train_eid = g.filter_edges(lambda edges: edges.data['train']).astype('int64')
g_train = g.edge_subgraph(train_eid, preserve_nodes=True)
g_train.copy_from_parent()
rating_train = g_train.edata['rating']
src_train, dst_train = g_train.all_edges()

## Construct the test set

Similarly, we use `filter_edges` to select the edge Ids for testing.

In [ ]:
eid_test = g.filter_edges(lambda edges: edges.data['test']).astype('int64')
src_test, dst_test = g.find_edges(eid_test)
rating_test = g.edges[eid_test].data['rating']

## Training

As above, training involves
1. Initializing a sampler
2. Iterating over the neighbor sampler, propagating the messages, and computing losses and gradients as usual.

In [ ]:
def train(g_train, src, dst, batch_size):
    # Training
    tot_loss = 0
    num_batches = 0
    for s, d, r, nodeflow in EdgeSampler(g_train, src, dst, rating_train):
        with mx.autograd.record():
            score = model.forward(nodeflow, s, d)
            loss = ((score - r) ** 2).mean()
            loss.backward()
        trainer.step(1)
        tot_loss += loss.asscalar()
        num_batches += 1
        
    # Return the training loss
    return tot_loss / num_batches

## Evaluation

To evaluate the performance of the trained model, we first compute node embeddings with GraphSage in mini-batches. Then we evaluate RMSE on the test set.

In [ ]:
def test(g, batch_size):
    # Validation & Test, we precompute GraphSage output for all nodes first.
    sampler = dgl.contrib.sampling.NeighborSampler(
        g,
        batch_size,
        num_neighbors,
        num_layers,
        seed_nodes=nd.arange(g.number_of_nodes()).astype('int64'),
        prefetch=True,
        add_self_loop=True,
        shuffle=False,
        num_workers=4
    )

    h = []
    for nf in sampler:
        h.append(model.gcn(nf))
    h = nd.concat(*h, dim=0)

    # Compute test RMSE
    node_biases = model.node_biases.data()
    score = (h[src_test] * h[dst_test]).sum(1) + node_biases[src_test + 1] + node_biases[dst_test + 1]
    test_rmse = nd.sqrt(((score - rating_test) ** 2).mean())
    
    return test_rmse.asscalar()

## Run the model

In [ ]:
model = GraphSAGERecommender(GraphSageWithSampling(100, 1, g_train))
model.collect_params().initialize(ctx=mx.cpu())
trainer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': 0.001, 'wd': 1e-9})

n_users = len(ml.user_ids)
n_products = len(ml.product_ids)

for epoch in range(200):
    loss = train(g_train, src_train, dst_train, batch_size)
    test_rmse = test(g, batch_size)
    print('Training loss:', loss, 'Test RMSE:', test_rmse)